In [2]:
import sys, os
sys.path.append(os.path.join(os.path.dirname('__file__'), '..', 'DB_and_Azure'))
import sql_db_functions as SQLf


### get data from server

In [3]:
conn, cursor = SQLf.sql_db_functions.connect_sql()

In [11]:
query = "SELECT product_characteristics.id, Brand_id , Detail, Brand FROM product_characteristics INNER JOIN Products ON product_characteristics.Brand_id = Products.Brand_Prod_id WHERE product_characteristics.encoded = false ;"
cursor.execute(query)

# Fetch the rows
rows = cursor.fetchall()
rows

[(58,
  57,
  '*Type: Shirt\n*Fit: Regular fit\n*Sleeve style: Long sleeve\n*Neckline: Button-down\n*Material: Tessuto fluido\n*Formality: Business Casual\n*Seasson: Spring, Autumn\n*Colors: White (90%), Black (10%)\n*Texture: Smooth\n*Transparency: Opaque\n*Details and Embellishments: Buttons, Front Pocket\n*Shape: Straight\n*Length: Hip-length\n*Collar Style: Button-down\n*Sleeve Style: Long\n*Patterns: Striped\n*Patterns placement: Throughout\n*Fluidity of fabric: High\n*Fabric weight: Light\n*Pocket Presence: Yes\n*Pocket placement: Left Chest\n*Pocket size: Medium\n*Breathability: High\n*Occasion Suitability: Business casual, Casual\n*Lapel: Not applicable',
  'Mango')]

In [4]:
SQLf.sql_db_functions.close_connection_db(conn=conn,cursor=cursor)

In [5]:
import pandas as pd

In [6]:
df = pd.DataFrame(rows ,columns= ['Id','prod_id','descripcion','Brand'])

In [7]:
df.head()

,Id,prod_id,descripcion,Brand
0,2,1,*Type: Shirt\n*Fit: Regular fit\n*Sleeve style...,Gucci
1,3,2,*Type: Shirt\n*Fit: Regular fit\n*Sleeve style...,Gucci
2,4,3,*Type: Top \n*Fit: Regular fit \n*Sleeve sty...,Gucci
3,5,4,*Type: Shirt \n*Fit: Regular fit \n*Sleeve s...,Gucci
4,6,5,*Type: Shirt \n*Fit: Regular fit \n*Sleeve s...,Gucci


In [8]:
df['descripcion'] = df['descripcion'].str.replace('\n*', ' / ')
df['descripcion'] = df['descripcion'].str.replace(r'\s+', ' ')
df['descripcion'] = df['descripcion'].str.replace('*', ' ')

df.head(5)

,Id,prod_id,descripcion,Brand
0,2,1,Type: Shirt / Fit: Regular fit / Sleeve style...,Gucci
1,3,2,Type: Shirt / Fit: Regular fit / Sleeve style...,Gucci
2,4,3,Type: Top / Fit: Regular fit / Sleeve sty...,Gucci
3,5,4,Type: Shirt / Fit: Regular fit / Sleeve s...,Gucci
4,6,5,Type: Shirt / Fit: Regular fit / Sleeve s...,Gucci


In [9]:
df['descripcion'] = "Brand: " + df['Brand'] + " /" + df['descripcion']
df.head()

,Id,prod_id,descripcion,Brand
0,2,1,Brand: Gucci / Type: Shirt / Fit: Regular fit ...,Gucci
1,3,2,Brand: Gucci / Type: Shirt / Fit: Regular fit ...,Gucci
2,4,3,Brand: Gucci / Type: Top / Fit: Regular fit ...,Gucci
3,5,4,Brand: Gucci / Type: Shirt / Fit: Regular fi...,Gucci
4,6,5,Brand: Gucci / Type: Shirt / Fit: Regular fi...,Gucci


In [10]:
df.drop(columns=['Brand'], axis=1, inplace=True)

In [11]:
documents = df['descripcion'].tolist()
id = df['Id'].astype('str').tolist()
metadata = [{"source": id} for id in df['Id'].astype('str').tolist()]

### Embeddings

In [12]:
#from langchain_chroma import Chroma

from langchain.vectorstores import Chroma

from langchain_community.embeddings.sentence_transformer import (
    SentenceTransformerEmbeddings,
)

In [13]:
embedding_function = SentenceTransformerEmbeddings(model_name='all-mpnet-base-v2')

c:\Users\juand\anaconda3\Lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  warn_deprecated(
c:\Users\juand\anaconda3\Lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
c:\Users\juand\anaconda3\Lib\site-packages\transformers\utils\generic.py:482: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
c:\

In [14]:
persist_directory = 'chroma-db'

In [28]:
db = Chroma.from_texts(texts=documents,
                  ids=id,
                  embedding=embedding_function,
                  collection_name='encodings',
                  metadatas=metadata,
                  persist_directory=persist_directory
                  )

In [29]:
db.similarity_search(query='hello')

[Document(page_content='Brand: HM / Type: Dress   / Fit: Regular fit   / Sleeve style: Long sleeve   / Neckline: V-neck   / Material: Cotton   / Formality: Casual   / Seasson: Autumn   / Colors: 100% Black   / Texture: Smooth   / Transparency: Opaque   / Details and Embellishments: Buttons   / Shape: Fitted   / Length: Midi   / Collar Style: Not applicable   / Sleeve Style: Long   / Patterns: Not applicable   / Patterns placement: Not applicable   / Fluidity of fabric: Moderate   / Fabric weight: Medium   / Pocket Presence: No   / Pocket placement: Not applicable   / Pocket size: Not applicable   / Breathability: Medium   / Occasion Suitability: Casual   / Lapel: Not applicable', metadata={'source': '19'}),
 Document(page_content='Brand: HM / Type: Dress / Fit: Regular fit / Sleeve style: Long sleeve / Neckline: V-neck / Material: Polyester / Formality: Casual / Season: Autumn, Spring / Colors: Black 90%, White 10% / Texture: Smooth / Transparency: Opaque / Details and Embellishments: 

In [30]:
db.persist()